##  数据预处理。

In [1]:
import os#与操作系统进行交互的库
print(f"为了上传到Kaggle平台,我对文件名进行了改变.根据年份应该能够知道4个文件的对应关系:\n{os.listdir('/kaggle/input/dcic-a/2025DCIC/')}")

为了上传到Kaggle平台,我对文件名进行了改变.根据年份应该能够知道4个文件的对应关系:
['A-X2014-2019.csv', 'A-y2014-2019.csv', 'example_submission.csv', 'A-X2020-2021.csv']


In [2]:
import pandas as pd#read csv,parquet
import numpy as np#for scientific computation of matrices
import warnings#avoid some negligible errors
#The filterwarnings () method is used to set warning filters, which can control the output method and level of warning information.
warnings.filterwarnings('ignore')

In [3]:
df=pd.concat([ pd.read_csv("/kaggle/input/dcic-a/2025DCIC/A-X2014-2019.csv"),
               pd.read_csv("/kaggle/input/dcic-a/2025DCIC/A-X2020-2021.csv")
          ])
print(f"df.shape:{df.shape}.\n df.nunique:{df.nunique()}")

df.shape:(4564507, 14).
 df.nunique:NAME        125
SENID       127
TIME     227120
V          3387
AVGV       2947
MAXV       1197
MAXT     588903
MINV       1204
MINT     588887
S             5
AVGS          5
MAXS          5
MINS          5
SPAN          1
dtype: int64


我们可以发现NAME和SENID不是一一对应关系,一个村子可以有2个SENID,同时SPAN只有唯一值,没有什么用.

我们先对数据按时间排序,然后填充缺失值,时间序列数据可以用前向填充,也就是填充上一个时刻的数值.

In [4]:
df=df.sort_values(['SENID','TIME'])
df=df.groupby('SENID')[df.columns].fillna(method='ffill').reset_index(drop=True)
df.head()

,NAME,SENID,TIME,V,AVGV,MAXV,MAXT,MINV,MINT,S,AVGS,MAXS,MINS,SPAN
0,下洋坂雨量,210254,2014-01-08 05:00:00,4.0,4.0,4.0,2014/1/8 5:00:00,4.0,2014/1/8 5:00:00,1.0,1.0,2.0,2.0,1
1,下洋坂雨量,210254,2014-01-13 22:00:00,1.0,1.0,1.0,2014/1/13 22:00:00,1.0,2014/1/13 22:00:00,1.0,1.0,2.0,2.0,1
2,下洋坂雨量,210254,2014-01-14 00:00:00,1.0,1.0,1.0,2014/1/14,1.0,2014/1/14,1.0,1.0,2.0,2.0,1
3,下洋坂雨量,210254,2014-01-14 01:00:00,1.0,1.0,1.0,2014/1/14 1:00:00,1.0,2014/1/14 1:00:00,1.0,1.0,2.0,2.0,1
4,下洋坂雨量,210254,2014-01-14 04:00:00,1.0,1.0,1.0,2014/1/14 4:00:00,1.0,2014/1/14 4:00:00,1.0,1.0,2.0,2.0,1


对于日期数据,我们可以用MAXT减去MINT. 考虑到MAXT减MINT为0的占比大于90%,我们可以直接把数据分为0和不为0这2种类别。

In [5]:
df['MAXT_MINT']=(pd.to_datetime(df['MAXT'],errors='coerce')-pd.to_datetime(df['MINT'],errors='coerce')).dt.seconds
print(f"nunique:{df['MAXT_MINT'].nunique()}")
print(f"MAXT_MINT为0的占比为:{len(df[df['MAXT_MINT']==0]*100)/len(df)}%.")

nunique:53
MAXT_MINT为0的占比为:0.9385773753879664%.


我们可以看到,很多特征表面上是连续型变量,我们其实也可以把它们当成类别型变量,进行简单的onehot编码之后再进行相关的操作。

In [6]:
for c in df.columns:
    if df[c].dtype!=object:
        most_v=df[c].value_counts().keys()[0]
        print(f"特征{c}的占比为:{len(df[df[c]==most_v])*100/len(df)}%")

特征SENID的占比为:4.585708818060746%
特征V的占比为:86.83139274405757%
特征AVGV的占比为:80.54845791670382%
特征MAXV的占比为:80.76911701526583%
特征MINV的占比为:80.66674013206683%
特征S的占比为:97.19538166991528%
特征AVGS的占比为:54.44169545582908%
特征MAXS的占比为:54.450655897778226%
特征MINS的占比为:54.44044669008066%
特征SPAN的占比为:100.0%
特征MAXT_MINT的占比为:93.85773753879663%


## 建模思路: 根据其他SENID在某个时刻的groupby特征去预测入库流量在这个时刻的V,所以目前就是要构造出更多表示某个时刻的特征.

## 特征工程.

下面以特征V举例来说明几种常见的特征工程。

## Z-score

In [7]:
df[f'V_mean']=df.groupby(['TIME'])['V'].transform('mean')
df[f'V_std']=df.groupby(['TIME'])['V'].transform('std')
df[f'V_zscore']=(df['V']-df[f'V_mean'])/(df[f'V_std']+1e-10)
df.head()

,NAME,SENID,TIME,V,AVGV,MAXV,MAXT,MINV,MINT,S,AVGS,MAXS,MINS,SPAN,MAXT_MINT,V_mean,V_std,V_zscore
0,下洋坂雨量,210254,2014-01-08 05:00:00,4.0,4.0,4.0,2014/1/8 5:00:00,4.0,2014/1/8 5:00:00,1.0,1.0,2.0,2.0,1,0.0,1.726229,1.210547,1.878302
1,下洋坂雨量,210254,2014-01-13 22:00:00,1.0,1.0,1.0,2014/1/13 22:00:00,1.0,2014/1/13 22:00:00,1.0,1.0,2.0,2.0,1,0.0,0.712068,0.450738,0.638802
2,下洋坂雨量,210254,2014-01-14 00:00:00,1.0,1.0,1.0,2014/1/14,1.0,2014/1/14,1.0,1.0,2.0,2.0,1,NaN,0.780227,0.486808,0.451457
3,下洋坂雨量,210254,2014-01-14 01:00:00,1.0,1.0,1.0,2014/1/14 1:00:00,1.0,2014/1/14 1:00:00,1.0,1.0,2.0,2.0,1,0.0,0.841294,0.572785,0.277077
4,下洋坂雨量,210254,2014-01-14 04:00:00,1.0,1.0,1.0,2014/1/14 4:00:00,1.0,2014/1/14 4:00:00,1.0,1.0,2.0,2.0,1,0.0,0.274946,0.422485,1.716163


虽然表面上只是进行了缩放,但是由于每个TIME的mean和std是不一样的,所以V_zscore和V还是有很大的区别的。

In [8]:
df[['V','V_zscore']].corr()

,V,V_zscore
V,1.000000,0.068874
V_zscore,0.068874,1.000000


## diff和shift特征

In [9]:
for gap in [1]:
    df['V_shift1']=df.groupby(['TIME'])['V'].shift(gap)
    df['V_diff1']=df.groupby(['TIME'])['V'].diff(gap)

## 滑动窗口

In [10]:
df[f'V_rolling3_mean']=df.groupby(['SENID'])['V'].transform(lambda x:x.rolling(window=3).mean())
df.head()

,NAME,SENID,TIME,V,AVGV,MAXV,MAXT,MINV,MINT,S,...,MAXS,MINS,SPAN,MAXT_MINT,V_mean,V_std,V_zscore,V_shift1,V_diff1,V_rolling3_mean
0,下洋坂雨量,210254,2014-01-08 05:00:00,4.0,4.0,4.0,2014/1/8 5:00:00,4.0,2014/1/8 5:00:00,1.0,...,2.0,2.0,1,0.0,1.726229,1.210547,1.878302,NaN,NaN,NaN
1,下洋坂雨量,210254,2014-01-13 22:00:00,1.0,1.0,1.0,2014/1/13 22:00:00,1.0,2014/1/13 22:00:00,1.0,...,2.0,2.0,1,0.0,0.712068,0.450738,0.638802,NaN,NaN,NaN
2,下洋坂雨量,210254,2014-01-14 00:00:00,1.0,1.0,1.0,2014/1/14,1.0,2014/1/14,1.0,...,2.0,2.0,1,NaN,0.780227,0.486808,0.451457,NaN,NaN,2.0
3,下洋坂雨量,210254,2014-01-14 01:00:00,1.0,1.0,1.0,2014/1/14 1:00:00,1.0,2014/1/14 1:00:00,1.0,...,2.0,2.0,1,0.0,0.841294,0.572785,0.277077,NaN,NaN,1.0
4,下洋坂雨量,210254,2014-01-14 04:00:00,1.0,1.0,1.0,2014/1/14 4:00:00,1.0,2014/1/14 4:00:00,1.0,...,2.0,2.0,1,0.0,0.274946,0.422485,1.716163,NaN,NaN,1.0


## 指数加权移动平均

In [11]:
df[f'V_ewm005_mean']=df.groupby(['SENID'])['V'].transform(lambda x:x.ewm(alpha=0.05).mean())
df.head()

,NAME,SENID,TIME,V,AVGV,MAXV,MAXT,MINV,MINT,S,...,MINS,SPAN,MAXT_MINT,V_mean,V_std,V_zscore,V_shift1,V_diff1,V_rolling3_mean,V_ewm005_mean
0,下洋坂雨量,210254,2014-01-08 05:00:00,4.0,4.0,4.0,2014/1/8 5:00:00,4.0,2014/1/8 5:00:00,1.0,...,2.0,1,0.0,1.726229,1.210547,1.878302,NaN,NaN,NaN,4.000000
1,下洋坂雨量,210254,2014-01-13 22:00:00,1.0,1.0,1.0,2014/1/13 22:00:00,1.0,2014/1/13 22:00:00,1.0,...,2.0,1,0.0,0.712068,0.450738,0.638802,NaN,NaN,NaN,2.461538
2,下洋坂雨量,210254,2014-01-14 00:00:00,1.0,1.0,1.0,2014/1/14,1.0,2014/1/14,1.0,...,2.0,1,NaN,0.780227,0.486808,0.451457,NaN,NaN,2.0,1.949167
3,下洋坂雨量,210254,2014-01-14 01:00:00,1.0,1.0,1.0,2014/1/14 1:00:00,1.0,2014/1/14 1:00:00,1.0,...,2.0,1,0.0,0.841294,0.572785,0.277077,NaN,NaN,1.0,1.693319
4,下洋坂雨量,210254,2014-01-14 04:00:00,1.0,1.0,1.0,2014/1/14 4:00:00,1.0,2014/1/14 4:00:00,1.0,...,2.0,1,0.0,0.274946,0.422485,1.716163,NaN,NaN,1.0,1.540078


## 当然也可以对上面几种情况进行组合,例如Zscore的滑动窗口、Zscore的指数加权移动平均.

我们可以看看它们的相关性,如果相关性越小,说明我们构造的特征越有效。我们也可以将这些操作应用于其他特征。

In [12]:
df[['V','V_zscore','V_shift1','V_diff1','V_rolling3_mean','V_ewm005_mean']].corr()

,V,V_zscore,V_shift1,V_diff1,V_rolling3_mean,V_ewm005_mean
V,1.000000,0.068874,0.003999,0.710102,0.965572,0.771453
V_zscore,0.068874,1.000000,0.000435,0.049101,0.059842,0.042402
V_shift1,0.003999,0.000435,1.000000,-0.701253,0.004089,0.004163
V_diff1,0.710102,0.049101,-0.701253,1.000000,0.685515,0.547069
V_rolling3_mean,0.965572,0.059842,0.004089,0.685515,1.000000,0.818844
V_ewm005_mean,0.771453,0.042402,0.004163,0.547069,0.818844,1.000000


## 我们的目的是构造每个时刻的特征,所以还要对刚才构造好的特征进行groupby操作.

In [13]:
STATS=['min','mean','std','max','median','skew','count','nunique']
g_col='TIME'
t_col='V'
agg_df = df[[g_col,t_col]].groupby(g_col).agg(STATS).reset_index()
agg_df.columns = ['_'.join(x) for x in agg_df.columns]
agg_df.columns = [ f'{g_col}_transform_{x}' for x in agg_df.columns if x!=g_col]
agg_df=agg_df.rename(columns={f"{g_col}_transform_{g_col}_":g_col})
agg_df.head()

,TIME,TIME_transform_V_min,TIME_transform_V_mean,TIME_transform_V_std,TIME_transform_V_max,TIME_transform_V_median,TIME_transform_V_skew,TIME_transform_V_count,TIME_transform_V_nunique
0,2014-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,24,1
1,2014-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,24,1
2,2014-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,24,1
3,2014-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,24,1
4,2014-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,24,1


##  下面到你了。